In [3]:
import networkx as nx
import dwave_networkx as dnx

from dqanneal.graph.embed import (get_problem_graph, 
                                  embed_problem_onto_hardware, 
                                  define_embedded_qubo_problem,
                                  unembed_samples,
                                  build_reverse_annealing_schedule_embedding)
from dqanneal.solvers.solvers import dimod_optimizer_cloud, save_annealing_results
import numpy as np

In [4]:
## quark and oeip embedding versions

# import quark
# import oeip
import importlib.metadata
print(importlib.metadata.version('quark'))
print(importlib.metadata.version('oeip'))

1.2
1.0


In [5]:
import dimod
import minorminer
import dwave # same as: dwave-ocean-sdk  
import dwaveoceansdk


print(dimod.__version__)
print(minorminer.__version__)
print(dwaveoceansdk.__version__)


0.12.18
0.2.17
8.2.0


In [6]:
from dwave import embedding

# Define the QUBO problem

In [7]:
dwave_qubo = dict()
for i in range(1,7):
    for j in range(i,7):
        dwave_qubo[(i,j)] = np.random.randint(1,10)

dwave_qubo


{(1, 1): 8,
 (1, 2): 3,
 (1, 3): 8,
 (1, 4): 7,
 (1, 5): 5,
 (1, 6): 3,
 (2, 2): 5,
 (2, 3): 1,
 (2, 4): 8,
 (2, 5): 9,
 (2, 6): 9,
 (3, 3): 9,
 (3, 4): 1,
 (3, 5): 4,
 (3, 6): 3,
 (4, 4): 4,
 (4, 5): 6,
 (4, 6): 3,
 (5, 5): 2,
 (5, 6): 8,
 (6, 6): 8}

In [8]:
# dwave_qubo = {val: key*1000 for val, key in dwave_qubo.items()}

In [9]:
source_bqm = dimod.BinaryQuadraticModel.from_qubo(dwave_qubo)
lin, quad, offset = source_bqm.to_ising()
source_ising = dimod.BinaryQuadraticModel.from_ising(lin, quad, offset)

print(f'numbera of variables: {len(source_ising.variables)}')

numbera of variables: 6


In [10]:
problem_graph, node_edge_dict= get_problem_graph(source_ising)
nx.get_edge_attributes(problem_graph, "bias")

{(2, 1): np.float64(0.75),
 (2, 3): np.float64(0.25),
 (2, 4): np.float64(2.0),
 (2, 5): np.float64(2.25),
 (2, 6): np.float64(2.25),
 (1, 3): np.float64(2.0),
 (1, 4): np.float64(1.75),
 (1, 5): np.float64(1.25),
 (1, 6): np.float64(0.75),
 (3, 4): np.float64(0.25),
 (3, 5): np.float64(1.0),
 (3, 6): np.float64(0.75),
 (4, 5): np.float64(1.5),
 (4, 6): np.float64(0.75),
 (5, 6): np.float64(2.0)}

In [11]:
from dqanneal.graph.embed import create_king_graph

hardware_graph, hardware_graph_edge_dict, pos = create_king_graph(4,4)
# nx.draw(hardware_graph,pos, with_labels = True)

In [12]:
embedded_problem, valid_flag = embed_problem_onto_hardware(problem_graph,
                                                            hardware_graph,
                                                            maxtime_sec=40,
                                                            attempts=10)


In [13]:
bool(valid_flag)

True

In [14]:
from dqanneal.graph.embed import opt_embedding_quark_ising
obj_true, obj_emd, bqm_emb = opt_embedding_quark_ising(embedded_problem,
                                                        hardware_graph,
                                                        lin,
                                                        quad,
                                                        offset
                                                        )


In [15]:
from quark import ScipModel
model = ScipModel.get_from_objective(obj_true)
solution = model.solve()
soln_dict = dict(solution.items())
soln_dict
solution.solving_status, solution.objective_value, soln_dict

('optimal', 0.0, {1: -1, 2: -1, 3: -1, 4: -1, 5: -1, 6: -1})

In [16]:
source_ising.energy(soln_dict)

np.float64(0.0)

In [17]:
exact = dimod.ExactSolver().sample_ising(lin, quad)
exact_out = exact.first.sample
source_ising.energy(exact_out)

np.float64(0.0)

In [18]:
exact_out == soln_dict

True

In [19]:
model_emb = ScipModel.get_from_objective(obj_emd)
solution_emb = model_emb.solve()
soln_dict_emb = dict(solution_emb.items())

## Put everything in the correct format
soln_dict_emb_sampleset = dimod.SampleSet.from_samples(soln_dict_emb, 
                                                    energy=[solution_emb.objective_value], 
                                                    vartype=dimod.vartypes.Vartype.SPIN 
                                                    )
converted_samples = embedding.unembed_sampleset(
                                soln_dict_emb_sampleset,
                                embedded_problem, 
                                source_ising)

converted_samples.first.sample == exact.first.sample

True